IDENTIFICADOR DE IDIOMA

https://regex101.com/

() - cria grupos de palavras (a|b)
[] - cria grupos de caracteres a[abc]c
- - intervalo 1-5
| - ou
* - 0 ou n vezes a repeticao [a-z]*
+ - n vezes a repeticao [a-z]+
{} - intervalo de n vezes de repeticao [a-z]{1,5}

^ - inicio da string ^g.* vai detectar tudo que começa com g

$ - fim da string $g.* vai detectar tudo que termina com g	


PEGAR OS DADOS

In [1]:
import pandas as pd
import re

from nltk.util import bigrams, everygrams
from nltk.lm.preprocessing import pad_both_ends, padded_everygram_pipeline
from nltk.tokenize import WhitespaceTokenizer
from nltk.lm import MLE, NgramCounter

from sklearn.model_selection import train_test_split



In [2]:
data_pt = pd.read_csv("data/stackoverflow_portugues.csv")
data_in = pd.read_csv("data/stackoverflow_ingles.csv")
data_es = pd.read_csv("data/stackoverflow_espanhol.csv" , delimiter=';')


TRATAR OS DADOS

In [3]:
def substituir_regex(data, regex, substituir = ''):
    if type(data) == str:
        return regex.sub(substituir,data)
    return [regex.sub(substituir, dat) for dat in data]

def regex_treatment(data):    
    regex_code = re.compile(r'<code>.*?</code>') # Remove partes com código
    regex_html = re.compile(r'<(.|(\n))*?>') # Remove as tags HTML
    regex_punctuation = re.compile(r'[^\w\s]') # Remove as pontuações
    regex_digit = re.compile(r'\d+') # Remove dígitos
    regex_line = re.compile(r'\n') # Remove quebra de linha

    # Realiza substituições sequenciais
    data = substituir_regex(data, regex_code, "---CODE---")
    data = substituir_regex(data, regex_html)
    data = substituir_regex(data, regex_punctuation)
    data = substituir_regex(data, regex_digit)
    data = substituir_regex(data, regex_line, ' ')

    # Remove espaços duplicados ao final
    regex_duplicate_space = re.compile(r'\s+')  # Normaliza múltiplos espaços
    data = substituir_regex(data, regex_duplicate_space, ' ')

    # Converte para minúsculas
    data = [txt.lower().strip() for txt in data]  # Remove espaços extras nas bordas

    return data

data_pt['regex_quest'] = regex_treatment(data_pt['Questão'])
data_es['regex_quest'] = regex_treatment(data_es['Questão'])
data_in['regex_quest'] = regex_treatment(data_in['Questão'])



In [4]:
print(data_pt['regex_quest'][0])
print(data_es['regex_quest'][0])
print(data_in['regex_quest'][0])

se eu fizer o hash de senhas antes de armazenálas em meu banco de dados é suficiente para evitar que elas sejam recuperadas por alguém estou falando apenas da recuperação diretamente do banco de dados e não qualquer outro tipo de ataque como força bruta na página de login da aplicação keylogger no cliente e criptoanálise rubberhose qualquer forma de hash não vai impedir esses ataques tenho preocupação em dificultar ou até impossibilitar a obtenção das senhas originais caso o banco de dados seja comprometido como dar maior garantia de segurança neste aspecto quais preocupações adicionais evitariam o acesso às senhas existem formas melhores de fazer esse hash
las sentencias dinámicas son sentencias sql que se crean como cadenas de texto strings y en las que se insertanconcatenan valores obtenidos de alguna fuente normalmente proveniente del usuario lo que puede hacer que sean vulnerables a inyección sql si no se sanean las entradas como por ejemplo id_usuario _postid mysql_queryselect fr

Model


In [5]:
def word_tokenize(data, ngram = 2):
    tokenizer = WhitespaceTokenizer()
    return padded_everygram_pipeline(ngram, tokenizer.tokenize(data))

def treinar_mle(txt, nmgram = 2):
    all_words, vocabulario = word_tokenize(' '.join(txt))

    model = MLE(nmgram)
    model.fit(all_words, vocabulario)
    return model

In [6]:
random_seed = 12
test_size = 0.2

pt_train, pt_test = train_test_split(data_pt['regex_quest'], random_state = random_seed, test_size=test_size)
es_train, es_test = train_test_split(data_es['regex_quest'], random_state = random_seed, test_size=test_size)
in_train, in_test = train_test_split(data_in['regex_quest'], random_state = random_seed, test_size=test_size)

In [7]:
model_pt = treinar_mle(pt_train)
model_es = treinar_mle(es_train)
model_in = treinar_mle(in_train)

In [35]:
def regex_data(data):
    regex_punctuation = re.compile(r'[^\w\s]') # Remove as pontuações
    regex_digit = re.compile(r'\d+') # Remove dígitos
    regex_line = re.compile(r'\n') # Remove quebra de linha

    # Realiza substituições sequenciais
    data = substituir_regex(data, regex_punctuation)
    data = substituir_regex(data, regex_digit)
    data = substituir_regex(data, regex_line, ' ')

    # Remove espaços duplicados ao final
    regex_duplicate_space = re.compile(r'\s+')  # Normaliza múltiplos espaços
    data = substituir_regex(data, regex_duplicate_space, ' ')
    return data

def get_bigrams(data):
    data = regex_data(data)
    tokenizer = WhitespaceTokenizer()
    tokens = tokenizer.tokenize(data.lower())

    fakechar = [list(pad_both_ends(palavra, n=2)) for palavra in tokens]
    return [list(bigrams(palavra)) for palavra in fakechar]

def test_perplexity(texto, model):
    return sum([model.perplexity(txt) for txt in get_bigrams(texto)])

test_perplexity(es_test.iloc[0], model_in)





inf